In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy
import seaborn as sns
from sklearn import linear_model
from sklearn.model_selection import train_test_split
import sklearn
from sklearn import tree
from IPython.display import Image
import pydotplus
import graphviz
from sklearn import ensemble
from sklearn.model_selection import cross_val_score
import time
import timeit
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Lasso
from sklearn.metrics import confusion_matrix
from sklearn.naive_bayes import BernoulliNB, GaussianNB
from nltk.corpus import stopwords
%matplotlib inline

In [2]:
df = pd.read_json(path_or_buf='Tools_and_Home_Improvement_5.json', orient='records', lines=True)

In [3]:
df['reviewText'] = df['reviewText'].str.lower()

In [4]:
df['reviewText'] = df['reviewText'].str.replace("@|\.|\+|<br>|<br />|/|\'|\"|\\|(|)|<p>|\\n|<|>|\?|#|[|]|%|\$|&|;|!|:|-|_|=|}|{|,",'')

In [5]:
def raterBin(x):
    if x >=3:
        return int(1)
    if x < 3:
        return int(0)

In [6]:
df['reviewBin'] = df['overall'].apply(raterBin)

In [7]:
pos_words = df.loc[(df['reviewBin'] == 1), ['reviewText','reviewBin']].reset_index()
del pos_words['index']

In [9]:
pos_words

,reviewText,reviewBin
0,i hate it when my shirt collars not otherwise ...,1
1,these little magnets are really powerful for t...,1
2,i wanted something this small to mount on the ...,1
3,i use these to magnetize my warhammer 40k mini...,1
4,they are soo freaking annoying why you spend ...,1
5,am using for 40k models they are a great size ...,1
6,the color pictures and exploded diagrams are a...,1
7,good simple projects to start you using the kr...,1
8,these are projects that people can do with pre...,1
9,if you have a pocket holl jig then this book i...,1


In [10]:
pos_words_sep = pd.Series(' '.join(df['reviewText']).lower().split()).value_counts()[:]
pos_words_sep = pos_words_sep.to_frame().reset_index()
pos_words_sep.columns = ['word','pos_count']



In [55]:
words = []
stopWords = set(stopwords.words('english'))
for w in pos_words_sep.word:
    if w not in stopWords and w != '2' and w != '3' and w != 'fit':
        words.append(w)
        
pos_words_final = pos_words_sep[pos_words_sep['word'].isin(words)]




In [56]:
pos_words_final = pos_words_final.iloc[:100]

In [57]:
pos_words_final.word

24            one
25          light
27            use
38           like
41          would
43           well
44          great
45           good
53            get
57           work
59           used
63           tool
64           time
66           much
68           also
69           dont
71           need
72           easy
74         little
75         really
77          works
78            saw
80            set
82          price
84           nice
85         bought
86            two
87          small
88          using
89           even
          ...    
148           old
149        switch
150          bulb
151          last
154           cut
155       plastic
156         think
158     recommend
159           box
160    flashlight
161          keep
162          many
163         house
164          sure
167          know
168          home
169       another
170           far
171           job
172         since
174         bulbs
176          size
178          case
179        doesnt
180       

In [58]:
for key in pos_words_final.word:
    df[str(key)] = df.reviewText.str.contains(str(key), case=False)
  

In [59]:
pd.set_option('display.max_columns',120)

In [60]:
df[pos_words_final.word]

,one,light,use,like,would,well,great,good,get,work,used,tool,time,much,also,dont,need,easy,little,really,works,saw,set,price,nice,bought,two,small,using,even,quality,made,bit,product,im,battery,better,ive,batteries,make,could,power,buy,enough,tools,long,blade,still,first,around,way,drill,back,led,put,lights,unit,thing,years,bright,new,got,want,see,without,go,lot,water,right,find,old,switch,bulb,last,cut,plastic,think,recommend,box,flashlight,keep,many,house,sure,know,home,another,far,job,since,bulbs,size,case,doesnt,fine,makes,needed,door,cant,problem
0,True,False,True,False,False,False,False,False,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,True,False,False,False,False,False,False,False
1,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,True,False,False,False,False,False,False,False,False,True,False,False,True,False,False,False,False,False,False,False,True,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False
2,False,False,False,False,False,True,False,False,False,True,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,True,False,False,False,False,True,False,True,False,False,False,False,False,True,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,True,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False
4,False,False,True,False,False,False,False,True,True,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,True,False,True,False,False,False,False,False,False,False,False,False,False,True,False,False,True,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
5,False,False,True,False,False,False,True,False,False,True,False,False,True,False,True,False,False,False,False,False,False,False,False,False,False,False,False,True,True,False,False,False,False,False,True,False,False,False,False,False,False,False,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,True,False,True,False,False,False,False,False,Fa

In [62]:
X = df[pos_words_final.word]
y = df['reviewBin']
X_train, X_test, y_train, y_test = train_test_split(X , y, test_size=.30)

In [63]:
bnb = BernoulliNB()

# Fit our model to the data.
bnb.fit(X_train, y_train)

# Classify, storing the result in a new variable.
y_pred = bnb.predict(X_test)

# Display our results.
print("Number of mislabeled points out of a total {} points : {}".format(
    X_train.shape[0],
    (y_test != y_pred).sum()
))

Number of mislabeled points out of a total 94133 points : 3623


tf-idf, count vectorizer